In [1]:
from bs4 import BeautifulSoup
import requests, json, re
from collections import defaultdict
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
req = requests.get("https://data.nola.gov/api/views/jsrk-e98x/rows.json?accessType=DOWNLOAD")

In [3]:
req

<Response [200]>

In [4]:
html = BeautifulSoup(req.text)

In [5]:
with open(r"C:\Users\iseedeadpeople\audio\data-maining text.txt", "w", encoding="utf8") as file:
    file.write(str(req.text))

In [6]:
with open(r"C:\Users\iseedeadpeople\audio\data-maining html.txt", "w", encoding="utf8") as file:
    file.write(str(html))

In [7]:
vhod = req.text.replace("\\n", " ").replace("\xa0", " ").replace("\xad", " ").lower()

In [8]:
result = re.findall(r'"([^"]{100,})"', vhod)

In [14]:
for q in range(10):
    print(result[q])
    print(" ")

this data comes from https://nola.nextrequest.com/ and is a feed of all public records requests that have been published. some requests may not be included because they contain sensitive or personally identifiable information.
 
...               re:     louisiana public records request                           the guste iii new affordable housing units project                           new orleans, louisiana       to whom it may concern:   this letter is a request for information from the city of new orleans, department of safety and permits (“dsp”) pursuant to the louisiana public records act, § 44:1 et seq. this request pertains to information related to the guste iii: new affordable housing units project located at 1302 simon bolivar avenue, new orleans, louisiana and bounded by south robertson street, clio street, simon bolivar avenue and thalia street (the “project”). the project is being developed by the housing authority of new orleans (“hano”).       in accordance with the lo

In [10]:

dic = {}
for sent in result:
    words = sent.split()
    for w in words:
        if w not in dic:
            dic[w] = {}
        for w2 in words:
            dic[w][w2]=1

In [11]:
#df = pd.DataFrame(dic)
#df.fillna(0, inplace=True)

In [12]:
#pca = PCA()
#res = pca.fit_transform(df)